In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input,ZeroPadding2D, BatchNormalization, AveragePooling2D, Add
from tensorflow.keras.models import Model
import pickle

In [4]:
!unzip /content/dogcatmini.zip -d /content/dogcatmini

Archive:  /content/dogcatmini.zip
replace /content/dogcatmini/cats_set/cat.4001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# layers
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
def convolutional_block(X, f, filters, stage, block, s=2): # Added block as an argument
  conv_name_base = 'res' + str(stage) + block + '_branch' # Now using block to form the name
  bn_name_base = 'bn' + str(stage) + block + '_branch'  # Now using block to form the name

  F1, F2, F3 = filters

  X_shortcut = X

  X = Conv2D(filters=F1, kernel_size=(1, 1), strides=s, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=F2, kernel_size=(f, f), strides=1, padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=F3, kernel_size=(1, 1), strides=1, padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

  X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=s, padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

def identity_block(X,f, filters, stage, block):
  conv_name_base='res'+str(stage)+block+'_branch'
  bn_name_base='bn'+str(stage)+block+'_branch'

  F1, F2, F3= filters
  X_shortcut=X

  X=Conv2D(filters=F1, kernel_size=(1,1), strides=1,padding='valid', name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F2, kernel_size=(f,f), strides=1,padding='same', name=conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F3, kernel_size=(1,1), strides=1,padding='valid', name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

  X=Add()([X,X_shortcut])
  X=Activation('relu')(X)

  return X

def ResNet50(input_shape=(224,224,3)):
  X_input=Input(input_shape)

  X=ZeroPadding2D((3,3))(X_input)

  X=Conv2D(64, (7,7), strides=(2,2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3, name='bn_conv1')(X)
  X=Activation('relu')(X)
  X=MaxPooling2D((3,3), strides=(2,2))(X)

  X=convolutional_block(X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
  X=identity_block(X, 3, [64,64,256], stage=2, block='b')
  X=identity_block(X, 3, [64,64,256], stage=2, block='c')

  X=convolutional_block(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
  X=identity_block(X, 3, [128,128,512], stage=3, block='b')
  X=identity_block(X, 3, [128,128,512], stage=3, block='c')
  X=identity_block(X, 3, [128,128,512], stage=3, block='d')

  X=convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
  X=identity_block(X, 3, [256,256,1024], stage=4, block='b')
  X=identity_block(X, 3, [256,256,1024], stage=4, block='c')
  X=identity_block(X, 3, [256,256,1024], stage=4, block='d')
  X=identity_block(X, 3, [256,256,1024], stage=4, block='e')
  X=identity_block(X, 3, [256,256,1024], stage=4, block='f')

  X=convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block='a', s=2)
  X=identity_block(X, 3, [512,512,2048], stage=5, block='b')
  X=identity_block(X, 3, [512,512,2048], stage=5, block='c')

  X=AveragePooling2D((2,2), name='avg_pool')(X)

  model=Model(inputs=X_input, outputs=X, name='ResNet50')

  return model


In [6]:
base_model = ResNet50(input_shape = (224, 224, 3))

In [15]:

NUMBER_OF_CLASSES=1

head_model = base_model.output

# Flatten the output of our model
head_model = Flatten()(head_model)

# Constructing fully connected layer
head_model = Dense(256, activation='relu', name='fc1', kernel_initializer=glorot_uniform(seed=0))(head_model)
head_model = Dense(128, activation='relu', name='fc2', kernel_initializer=glorot_uniform(seed=0))(head_model)
# For binary classification
head_model = Dense(1, activation='sigmoid', name='fc3', kernel_initializer=glorot_uniform(seed=0))(head_model)


model = Model(
    inputs = base_model.input,
    outputs = head_model
)

In [16]:

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1 (Conv2D)            │ (None, 112, 112, 64)   │          9,472 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_conv1                  │ (None, 112, 112, 64)   │            256 │ conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 112, 112, 64)   │              0 │ bn_conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 55, 55, 64)     │              0 │ activation[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2a (Conv2D)   │ (None, 55, 55, 64)     │          4,160 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2a             │ (None, 55, 55, 64)     │            256 │ res2a_branch2a[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 55, 55, 64)     │              0 │ bn2a_branch2a[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2b (Conv2D)   │ (None, 55, 55, 64)     │         36,928 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2b             │ (None, 55, 55, 64)     │            256 │ res2a_branch2b[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 55, 55, 64)     │              0 │ bn2a_branch2b[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2c (Conv2D)   │ (None, 55, 55, 256)    │         16,640 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch1 (Conv2D)    │ (None, 55, 55, 256)    │         16,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2c             │ (None, 55, 55, 256)    │          1,024 │ res2a_branch2c[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch1              │ (None, 55, 55, 256)    │          1,024 │ res2a_branch1[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 28,339,585 (108.11 MB)

 Trainable params: 28,286,465 (107.90 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [14]:
import tensorflow as tf

# Define the preprocessing function
def preprocess(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Ensure 3 channels
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# List all image files
data = tf.data.Dataset.list_files('/content/dogcatmini/cats_set/*.jpg', shuffle=False)
data = data.concatenate(tf.data.Dataset.list_files('/content/dogcatmini/dogs_set/*.jpg', shuffle=False))

# Create labels: 0 for cats, 1 for dogs
labels_cats = tf.data.Dataset.from_tensor_slices(tf.zeros(tf.data.experimental.cardinality(tf.data.Dataset.list_files('/content/dogcatmini/cats_set/*.jpg')).numpy(), dtype=tf.int8))
labels_dogs = tf.data.Dataset.from_tensor_slices(tf.ones(tf.data.experimental.cardinality(tf.data.Dataset.list_files('/content/dogcatmini/dogs_set/*.jpg')).numpy(), dtype=tf.int8))
labels = labels_cats.concatenate(labels_dogs)

# Combine images and labels
data = tf.data.Dataset.zip((data, labels))

# Apply preprocessing
data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle, batch, and prefetch
data = data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Split into training and validation sets
total_count = tf.data.experimental.cardinality(data).numpy()
train_count = int(total_count * 0.8)
train_data = data.take(train_count)
validation_data = data.skip(train_count)


In [25]:
import tensorflow as tf

# List all image files for cats and dogs
cats_files = tf.data.Dataset.list_files('/content/dogcatmini/cats_set/*.jpg', shuffle=False)
dogs_files = tf.data.Dataset.list_files('/content/dogcatmini/dogs_set/*.jpg', shuffle=False)

# Get the number of files
num_cats = tf.data.experimental.cardinality(cats_files).numpy()
num_dogs = tf.data.experimental.cardinality(dogs_files).numpy()

# Print the lengths
print(f"Number of cat images: {num_cats}")
print(f"Number of dog images: {num_dogs}")


Number of cat images: 500
Number of dog images: 500


In [27]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint = ModelCheckpoint(
    filepath='model_checkpoint.h5',  # Save path for the model
    monitor='val_loss',             # Monitor validation loss
    save_best_only=True,            # Save only the best model
    save_weights_only=False,        # Save the full model (set to True for weights only)
    verbose=1                       # Print a message when saving
)

# Train the model with the checkpoint
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=20,
    callbacks=[checkpoint]
)


In [ ]:
model.fit(train_data, epochs=30, validation_data=validation_data)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 75s 609ms/step - accuracy: 0.7923 - loss: 0.4242 - val_accuracy: 0.8000 - val_loss: 0.4470
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 377ms/step - accuracy: 0.8657 - loss: 0.2999 - val_accuracy: 0.8700 - val_loss: 0.3206
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 365ms/step - accuracy: 0.9018 - loss: 0.2409 - val_accuracy: 0.8400 - val_loss: 0.3571
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 400ms/step - accuracy: 0.9442 - loss: 0.1476 - val_accuracy: 0.8950 - val_loss: 0.2154
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 374ms/step - accuracy: 0.9460 - loss: 0.1451 - val_accuracy: 0.8800 - val_loss: 0.2657
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 369ms/step - accuracy: 0.9626 - loss: 0.1032 - val_accuracy: 0.9100 - val_loss: 0.2102
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 394ms/step - accuracy: 0.9684 - loss: 0.0886 - val_accuracy: 0.9800 - val_loss: 0.0985
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 367ms/step - accuracy: 0.9746 - loss: 0.0502 - val_accu

In [24]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model_checkpoint.h5')

# Resume training or evaluate
model.fit(train_data, validation_data=validation_data, epochs=10)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predictions: [[0.94906545]
 [0.3695187 ]
 [0.5587614 ]
 [0.41375753]
 [0.45233116]
 [0.31561577]
 [0.6216028 ]
 [0.50914836]
 [0.68915844]
 [0.3551591 ]]
Labels: [1 1 0 0 1 0 0 0 0 1]


In [ ]:
# Save weights
model.save_weights('model_weights.h5')

# Load weights into the same model architecture
model.load_weights('model_weights.h5')